In [ ]:
import tensorflow as tf
import importlib
import os
import loss_functions
import dataprocessor
import fd_model
import change_point_detection
from tqdm import tqdm

def reload_custom_libs():
    importlib.reload(loss_functions)
    importlib.reload(dataprocessor)
    importlib.reload(change_point_detection)
    importlib.reload(fd_model)
    
from change_point_detection import *
from loss_functions import *
from fd_model import *
from dataprocessor import *

macd_timescales = [(8, 24), (16, 28), (32, 96)]
rtn_timescales = [1, 21, 63, 126, 252]
timesteps = 126
folder_path = "data"
files = [f for f in os.listdir(folder_path) if f.endswith(".xlsx")]
files = [
    file
    for file in files
    if file
    not in (
        "CC00.NYB.xlsx",
        "LB00.CME.xlsx",
        "ES00.CME.xlsx",
        "NQ00.CME.xlsx",
        "YM00.CBT.xlsx",
        "SP00.CME.xlsx",
    )
]

In [306]:
# 加载文件
# data_list = process_data_list(files, macd_timescales, rtn_timescales, test=125)
data_list = process_data_list(files, macd_timescales, rtn_timescales)

处理文件中。。: 100%|██████████| 11/11 [00:57<00:00,  5.25s/it]


In [ ]:
# # 跑过一次的话，不要再跑入这个

# gaussion_process_list = gaussion_process_list
# gaussion_process_list = []
# for data in tqdm(data_list):
#     price_series = data["close"]
#     target = price_series.to_numpy().reshape((-1, 1))
#     segment_list = get_segment_points(target, l_max=63, mu=0.999)
#     segment_list = [data.iloc[start : end, :] for start, end in segment_list]
#     gaussion_process_list.extend(segment_list)

# 模型预测

In [316]:
def reload_custom_libs():
    importlib.reload(loss_functions)
    importlib.reload(dataprocessor)
    importlib.reload(change_point_detection)
    importlib.reload(fd_model)
    
from change_point_detection import *
from loss_functions import *
from fd_model import *
from dataprocessor import *

In [317]:
# 生成 target_set 和 context_set
asset_num, context_num = len(data_list), 2
target_set, labels, map = generate_tensors(data_list, time_steps=timesteps, encoder_type = "one-hot", return_map=True)
target_set, context_set, labels = gaussian_data_binder(
    data_list,
    target_set,
    labels,
    map=map,
    asset_num=2,
    context_num=2,
    gaussion_process_list=gaussion_process_list,
)

# 设置参数
target_std = tf.cast(5e-2, tf.float64)
hidden_dim = 64  # 128
warm_up = 63
features_len = len(macd_timescales) + len(rtn_timescales)

# declare 数据, 初始化数据集
x, s = target_set[0], target_set[-1]
x_c_rtn, x_c, s_c = context_set[0], context_set[0][:, :, :, 1:], context_set[-1]
dataset = tf.data.Dataset.from_tensor_slices((x_c, x_c_rtn, s_c, x, s, labels))

timesteps = x.shape[-2]
features_len = x.shape[-1]
encoding_size = s.shape[-1]
x_shape = (None, asset_num, timesteps, features_len)
s_shape = (None, asset_num, timesteps, encoding_size)

生成张量, 并对类别信息进行one-hot 编码: 100%|██████████| 11/11 [00:47<00:00,  4.31s/it]


one-hot 编码中...


# 训练模型

In [318]:
# 训练模型
def train(model, dataset: tf.data.Dataset, batch_num: int, num_epochs: int, alpha: float):

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    dataset = dataset.shuffle(buffer_size=10000).batch(batch_num)

    for epoch in range(num_epochs):
        iter_count = 0
        for batch_data in tqdm(dataset, desc=f"训练中...epoch{epoch}"):
            # 对每一个批次进行处理
            x_c, x_c_rtn, s_c, x, s, labels = batch_data
            with tf.GradientTape() as tape:
                result = model(x_c, x_c_rtn, s_c, x, s)
                joint_loss, mle, sharpe = joint_loss_function(
                    result, labels, target_std, warm_up, alpha=alpha
                )
            grads = tape.gradient(joint_loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

            iter_count += 1
            if iter_count % 1 == 0 and False:
                print(
                    f"Epoch {epoch+1}/{num_epochs}, Iteration {iter_count}, Loss: {joint_loss.numpy()}, MLE: {mle.numpy()}, Sharpe: {sharpe.numpy()}"
                )

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {joint_loss.numpy()}")

# 初始化模型
model = ModelWrapper(features_len, hidden_dim, encoding_size, num_heads=4)
model.build((x_shape, s_shape))
result = train(model, dataset, batch_num=64, num_epochs=100, alpha = 1e-2)

训练中...epoch0: 100%|██████████| 57/57 [04:02<00:00,  4.25s/it]


Epoch 1/100, Loss: -0.2503902066794648


训练中...epoch1: 100%|██████████| 57/57 [04:57<00:00,  5.23s/it]


Epoch 2/100, Loss: 0.20365322097569394


训练中...epoch2: 100%|██████████| 57/57 [05:14<00:00,  5.51s/it]


...

训练中...epoch17: 100%|██████████| 57/57 [06:14<00:00,  6.58s/it]


Epoch 18/100, Loss: 0.0586711740506618


训练中...epoch18: 100%|██████████| 57/57 [06:40<00:00,  7.03s/it]


Epoch 19/100, Loss: -0.39565746607714664


训练中...epoch19:  42%|████▏     | 24/57 [02:48<03:48,  6.94s/it]